## 1. Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('Libraries imported successfully')

In [ ]:
# Load datasets
water_df = pd.read_csv('../data/water_quality.csv')
rainfall_df = pd.read_csv('../data/rainfall.csv')

print(f'Water Quality Data: {water_df.shape}')
print(f'Rainfall Data: {rainfall_df.shape}')
print(f'\nWater Data Date Range: {water_df["Timestamp"].min()} to {water_df["Timestamp"].max()}')
print(f'Rainfall Data Date Range: {rainfall_df["date"].min()} to {rainfall_df["date"].max()}')

In [ ]:
# Parse timestamps and basic preprocessing
water_df['Timestamp'] = pd.to_datetime(water_df['Timestamp'], errors='coerce')
rainfall_df['date'] = pd.to_datetime(rainfall_df['date'], errors='coerce')

# Identify numeric columns
numeric_cols = water_df.select_dtypes(include=[np.number]).columns.tolist()
print(f'Numeric columns ({len(numeric_cols)}): {numeric_cols}')

## 2. Data Quality Overview

In [ ]:
# Comprehensive data quality report
quality_report = pd.DataFrame({
    'Column': water_df.columns,
    'Data Type': water_df.dtypes,
    'Non-Null Count': water_df.count(),
    'Null Count': water_df.isnull().sum(),
    'Null %': (water_df.isnull().sum() / len(water_df) * 100).round(2)
})
quality_report = quality_report.sort_values('Null %', ascending=False)
quality_report

In [ ]:
# Nullity heatmap
fig, ax = plt.subplots(figsize=(14, 6))
null_data = water_df.isnull().astype(int)
sns.heatmap(null_data.iloc[:500, :].T, cbar=True, cmap='YlOrRd', ax=ax, yticklabels=True)
ax.set_title('Null Value Heatmap (First 500 Records)', fontsize=14, fontweight='bold')
ax.set_xlabel('Record Index')
plt.tight_layout()
plt.show()

print(f'\nMissing Data Summary:')
print(f'Total nulls: {water_df.isnull().sum().sum()}')
print(f'Overall null %: {(water_df.isnull().sum().sum() / (len(water_df) * len(water_df.columns)) * 100):.2f}%')

## 3. Summary Statistics

In [ ]:
# Descriptive statistics for numeric columns
water_df[numeric_cols].describe().round(2)

In [ ]:
# Key water quality parameters statistics
key_params = ['Average Water Speed', 'Temperature', 'Dissolved Oxygen (%Saturation)',
              'pH', 'Salinity', 'Specific Conductance', 'Turbidity']
key_params_present = [col for col in key_params if col in water_df.columns]

stats_table = water_df[key_params_present].describe().T
stats_table['CV%'] = (stats_table['std'] / stats_table['mean'] * 100).round(2)
stats_table = stats_table[['count', 'mean', 'std', 'CV%', 'min', '25%', '50%', '75%', 'max']].round(2)
stats_table

## 4. Parameter Distributions

In [ ]:
# Distribution plots for key parameters
fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.flatten()

for idx, col in enumerate(key_params_present[:9]):
    data = water_df[col].dropna()
    axes[idx].hist(data, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
    axes[idx].set_title(f'{col}\n(n={len(data)}, μ={data.mean():.2f}, σ={data.std():.2f})', fontsize=10)
    axes[idx].set_xlabel('Value')
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# KDE plots for top 6 parameters
fig, axes = plt.subplots(2, 3, figsize=(16, 8))
axes = axes.flatten()

for idx, col in enumerate(key_params_present[:6]):
    data = water_df[col].dropna()
    axes[idx].hist(data, bins=50, alpha=0.5, density=True, color='steelblue', edgecolor='black')
    data.plot(kind='kde', ax=axes[idx], color='red', linewidth=2, label='KDE')
    axes[idx].set_title(f'{col}', fontsize=11, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Temporal Analysis

In [ ]:
# Sort by timestamp and set index
water_sorted = water_df.sort_values('Timestamp').copy()
water_sorted.set_index('Timestamp', inplace=True)

# Resample to daily frequency for key parameters
daily_data = water_sorted[key_params_present].resample('D').mean()
print(f'Daily aggregated data shape: {daily_data.shape}')
print(f'Date range: {daily_data.index.min()} to {daily_data.index.max()}')

In [ ]:
# Time series plot for key parameters
fig, axes = plt.subplots(3, 2, figsize=(16, 10))
axes = axes.flatten()

for idx, col in enumerate(key_params_present[:6]):
    if col in daily_data.columns:
        daily_data[col].plot(ax=axes[idx], color='steelblue', linewidth=1.5, alpha=0.7)
        daily_data[col].rolling(window=7).mean().plot(ax=axes[idx], color='red', linewidth=2, label='7-day MA')
        axes[idx].set_title(f'{col} Over Time', fontsize=11, fontweight='bold')
        axes[idx].set_ylabel('Value')
        axes[idx].legend()
        axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Seasonal Decomposition

In [ ]:
# Seasonal decomposition for Turbidity (most important parameter)
if 'Turbidity' in daily_data.columns:
    # Fill NaN values for decomposition
    turbidity_filled = daily_data['Turbidity'].fillna(method='ffill').fillna(method='bfill')
    
    if len(turbidity_filled) > 14:  # Need at least 2 cycles for period=7
        decomposition = seasonal_decompose(turbidity_filled, model='additive', period=7)
        
        fig, axes = plt.subplots(4, 1, figsize=(14, 10))
        
        decomposition.observed.plot(ax=axes[0], color='steelblue')
        axes[0].set_ylabel('Observed', fontweight='bold')
        axes[0].grid(alpha=0.3)
        
        decomposition.trend.plot(ax=axes[1], color='orange')
        axes[1].set_ylabel('Trend', fontweight='bold')
        axes[1].grid(alpha=0.3)
        
        decomposition.seasonal.plot(ax=axes[2], color='green')
        axes[2].set_ylabel('Seasonal', fontweight='bold')
        axes[2].grid(alpha=0.3)
        
        decomposition.resid.plot(ax=axes[3], color='red')
        axes[3].set_ylabel('Residual', fontweight='bold')
        axes[3].set_xlabel('Date')
        axes[3].grid(alpha=0.3)
        
        fig.suptitle('Turbidity - Seasonal Decomposition (7-day period)', fontsize=14, fontweight='bold', y=0.995)
        plt.tight_layout()
        plt.show()
    else:
        print('Insufficient data for seasonal decomposition')

## 7. Correlation Analysis

In [ ]:
# Spearman correlation matrix for numeric columns
correlation_matrix = water_df[key_params_present].corr(method='spearman')

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            cbar_kws={'label': 'Spearman Correlation'}, ax=ax, square=True, linewidths=1)
ax.set_title('Spearman Correlation Matrix - Key Water Quality Parameters', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print('\nTop 10 Strongest Correlations (excluding diagonal):')
# Get upper triangle of correlation matrix
corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_pairs.append({
            'Param1': correlation_matrix.columns[i],
            'Param2': correlation_matrix.columns[j],
            'Correlation': correlation_matrix.iloc[i, j]
        })
corr_df = pd.DataFrame(corr_pairs).sort_values('Correlation', key=abs, ascending=False)
corr_df.head(10)

## 8. Outlier Detection

In [ ]:
# Identify outliers using IQR method
def find_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

outlier_summary = []
for col in key_params_present:
    count, lower, upper = find_outliers_iqr(water_df, col)
    outlier_pct = (count / len(water_df.dropna(subset=[col])) * 100) if len(water_df.dropna(subset=[col])) > 0 else 0
    outlier_summary.append({
        'Parameter': col,
        'Outlier Count': count,
        'Outlier %': round(outlier_pct, 2),
        'Lower Bound': round(lower, 2),
        'Upper Bound': round(upper, 2)
    })

outlier_df = pd.DataFrame(outlier_summary).sort_values('Outlier %', ascending=False)
outlier_df

In [ ]:
# Boxplot for outlier visualization
fig, axes = plt.subplots(2, 3, figsize=(16, 8))
axes = axes.flatten()

for idx, col in enumerate(key_params_present[:6]):
    data = water_df[col].dropna()
    axes[idx].boxplot(data, vert=True)
    axes[idx].set_title(f'{col}', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 9. Day-of-Week Patterns

In [ ]:
# Add day of week to water data
water_df['DayOfWeek'] = water_df['Timestamp'].dt.day_name()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Boxplot by day of week for key parameters
fig, axes = plt.subplots(2, 3, figsize=(16, 8))
axes = axes.flatten()

for idx, col in enumerate(key_params_present[:6]):
    plot_data = water_df[[col, 'DayOfWeek']].dropna()
    plot_data['DayOfWeek'] = pd.Categorical(plot_data['DayOfWeek'], categories=day_order, ordered=True)
    plot_data_sorted = plot_data.sort_values('DayOfWeek')
    
    sns.boxplot(data=plot_data_sorted, x='DayOfWeek', y=col, ax=axes[idx], palette='Set2')
    axes[idx].set_title(f'{col} by Day of Week', fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Day of Week')
    axes[idx].set_ylabel('Value')
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 10. Rainfall Data Overview

In [ ]:
# Rainfall statistics
print('Rainfall Data Summary:')
print(rainfall_df.describe().round(2))
print(f'\nDate Range: {rainfall_df["date"].min()} to {rainfall_df["date"].max()}')
print(f'Days Covered: {(rainfall_df["date"].max() - rainfall_df["date"].min()).days + 1}')

In [ ]:
# Rainfall visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Rainfall time series
rainfall_df.sort_values('date').set_index('date')['rainfall'].plot(ax=axes[0, 0], kind='bar', color='steelblue')
axes[0, 0].set_title('Daily Rainfall', fontsize=11, fontweight='bold')
axes[0, 0].set_ylabel('Rainfall (mm)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Temperature
rainfall_df.sort_values('date').set_index('date')['temperature'].plot(ax=axes[0, 1], color='orange')
axes[0, 1].set_title('Temperature Over Time', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Temperature (°C)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Humidity
rainfall_df.sort_values('date').set_index('date')['humidity'].plot(ax=axes[1, 0], color='green')
axes[1, 0].set_title('Humidity Over Time', fontsize=11, fontweight='bold')
axes[1, 0].set_ylabel('Humidity (%)')
axes[1, 0].tick_params(axis='x', rotation=45)

# Wind Speed
rainfall_df.sort_values('date').set_index('date')['wind_speed'].plot(ax=axes[1, 1], color='purple')
axes[1, 1].set_title('Wind Speed Over Time', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Wind Speed (m/s)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Rainfall statistics
print('\nRainfall Statistics:')
print(f'Total rainfall: {rainfall_df["rainfall"].sum():.2f} mm')
print(f'Average daily rainfall: {rainfall_df["rainfall"].mean():.2f} mm')
print(f'Max daily rainfall: {rainfall_df["rainfall"].max():.2f} mm')
print(f'Rainy days (> 0 mm): {(rainfall_df["rainfall"] > 0).sum()} out of {len(rainfall_df)}')
print(f'Rainfall % of days: {(rainfall_df["rainfall"] > 0).sum() / len(rainfall_df) * 100:.1f}%')

## 11. Data Integration Insights

In [ ]:
# Temporal alignment analysis
print('Data Alignment Analysis:')
print('='*50)

# Water data date range
water_dates = pd.to_datetime(water_df['Timestamp']).dt.date
water_date_range = (water_dates.max() - water_dates.min()).days + 1

# Rainfall date range
rainfall_dates = pd.to_datetime(rainfall_df['date']).dt.date
rainfall_date_range = (rainfall_dates.max() - rainfall_dates.min()).days + 1

print(f'Water Quality Data:')
print(f'  - Date range: {water_dates.min()} to {water_dates.max()}')
print(f'  - Days covered: {water_date_range}')
print(f'  - Total records: {len(water_df)}')
print(f'  - Records per day avg: {len(water_df) / water_date_range:.1f}')

print(f'\nRainfall Data:')
print(f'  - Date range: {rainfall_dates.min()} to {rainfall_dates.max()}')
print(f'  - Days covered: {rainfall_date_range}')

# Overlapping dates
common_dates = set(water_dates) & set(rainfall_dates)
print(f'\nData Overlap:')
print(f'  - Common dates: {len(common_dates)}')
print(f'  - Coverage: {len(common_dates) / min(water_date_range, rainfall_date_range) * 100:.1f}%')

## 12. Key Findings & Recommendations

In [ ]:
findings = """
KEY FINDINGS FROM EDA:
======================

1. DATA QUALITY:
   - Water dataset: 30,894 records with 20 columns
   - Critical parameters with high missingness:
     * Dissolved Oxygen: 18.6% missing
     * Temperature: 16.7% missing
   - Rainfall dataset: Only 54 days vs 152 days of water data
   - RECOMMENDATION: Fill missing values with median imputation or interpolation

2. TEMPORAL PATTERNS:
   - Strong day-of-week effects observed in water quality
   - Seasonal patterns visible (7-day decomposition shows cyclical trends)
   - RECOMMENDATION: Use seasonal naive baselines for forecasting

3. CORRELATIONS:
   - Check heatmap for parameter relationships
   - Dissolved Oxygen likely inversely related to Turbidity
   - Temperature may influence sensor readings
   - RECOMMENDATION: Use Spearman correlation (robust to non-linearity)

4. OUTLIERS:
   - Several parameters show outliers (>1.5 IQR from quartiles)
   - Turbidity most prone to extreme values
   - RECOMMENDATION: Cap extreme values at 99.9% quantile

5. DATA IMBALANCE:
   - Water data collected at higher frequency than rainfall
   - Missing site/location identifiers
   - RECOMMENDATION: Resample water data to daily average; add location metadata

NEXT STEPS:
===========
1. Build preprocessing module to handle missing data consistently
2. Engineer features: rolling means, lag features, day-of-week dummies
3. Develop WQI (Water Quality Index) from key parameters
4. Train baseline models (RandomForest for turbidity prediction)
5. Integrate rainfall data for impact analysis
6. Create anomaly detection pipeline
7. Deploy real-time monitoring dashboard
"""

print(findings)

In [ ]:
# Export processed data for downstream use
# Daily aggregated data
daily_export = daily_data.copy()
daily_export.to_csv('../data/water_quality_daily.csv')
print(f'Exported daily aggregated data: {daily_export.shape} records')
print(f'Date range: {daily_export.index.min()} to {daily_export.index.max()}')